In [ ]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

bin/
bin/train-tree-tagger
bin/separate-punctuation
bin/tree-tagger
bin/tree-tagger-flush
cmd/
cmd/lookup.perl
COPYRIGHT
doc/
doc/sigdat95.pdf
doc/nemlap94.pdf
FILES
README
Release-Notes

English parameter file installed.
Tagging scripts installed.
Path variables modified in tagging scripts.

You might want to add /content/treetagger/cmd and /content/treetagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for treetaggerwrapper: filename=treetaggerwrapper-2.3-py3-none-any.whl size=40759 sha256=0cc475021dffc44e6ee800fda6913c8bc27dc69ee057d21e05c25adf51480c4e
  Stored in directory: /root/.cache/pip/wheels/ea/d5/4b/a29ceaa48c687208c69a791394c02c8e432971a98d8e5fc9ca
Successfully built treetaggerwrapper


--2023-08-16 03:56:36--  https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
Resolving cis.lmu.de (cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to cis.lmu.de (cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889240 (1.8M) [application/x-gzip]
Saving to: ‘tree-tagger-linux-3.2.4.tar.gz’

     0K .......... .......... .......... .......... ..........  2%  118K 15s
    50K .......... .......... .......... .......... ..........  5%  236K 11s
   100K .......... .......... .......... .......... ..........  8% 29.4M 7s
   150K .......... .......... .......... .......... .......... 10% 30.9M 5s
   200K .......... .......... .......... .......... .......... 13%  238K 5s
   250K .......... .......... .......... .......... .......... 16% 28.7M 4s
   300K .......... .......... .......... .......... .......... 18% 24.6M 4s
   350K .......... .......... .......... .......... .......... 21% 21.4M 3s
   400

In [ ]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
!pip install nltk
import nltk
import string
import pandas as pd
import treetaggerwrapper as tt
nltk.download('punkt')  # Download the necessary tokenizer data if not already present
nltk.download('wordnet')
import spacy
# import neuralcoref

/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def preprocess_text(text):
  '''
  Tokenizes the text into sentences, removes punctuation from each sentence, and returns whole text (str) as a list of sentences
  '''
  # Tokenize the text into sentences
  sentences = sent_tokenize(text)
  # Remove punctuation from each sentence
  translator = str.maketrans('', '', string.punctuation)
  preprocessed_sentences = [sentence.translate(translator) for sentence in sentences]
  return preprocessed_sentences

def semantic_tag_sentence(sentence):
  '''
  Generates semantic tags for all words in 1 sentence

  Args:
    sentence (str): Input sentence

  Output:
    All tags as a list
  '''
  semantic_tags = []
  for word in word_tokenize(sentence):
    if(len(wordnet.synsets(word)) > 0):
      a = lesk(word_tokenize(sentence),word)
      semantic_tags.append(a)
    else:
      semantic_tags.append("")
  return semantic_tags

def semantic_tag_text(text):
  '''
  Takes in text (str) and returns a list with all semantic tags
  '''
  semantic_tags_text = []
  for sentence in text:
    semantic_tags_text.append(semantic_tag_sentence(sentence))
  return semantic_tags_text

def lemmatizer_tagger(sentence, TAGDIR = 'treetagger/', debug = False):
  """
  Takes in sentences (or any text) and returns three lists: original, lemmas and tags.

  Args:
    sentence (str): Input text to be tagged
    TAGDIR (str): location of directory with installation of treetaggerwrapper library
    debug (bool): If true, prints all the processed stuff as a pandas dataframe

  Output:
    A list comprising of 3 sublists, each one of them comprised of tokenized words of original sentence, lemmatized sentence and tags respectively
  """
  t_tagger = tt.TreeTagger(TAGLANG='en', TAGDIR=TAGDIR)
  pos_tags = t_tagger.tag_text(sentence)

  original = []
  lemmas = []
  tags = []
  for t in pos_tags:
	  original.append(t.split('\t')[0])
	  tags.append(t.split('\t')[1])
	  lemmas.append(t.split('\t')[-1])

  if debug:
    Results = pd.DataFrame({'Original': original, 'Lemma': lemmas, 'Tags': tags})
    print(Results)

  return original, lemmas, tags

def lem_tag_text(text):
  '''
  Takes whole text (str) as input, and lemmatizes and tags each sentence to output three 2D lists - original, cleaned and tags
  '''
  original = []
  lemmas = []
  tags = []
  for sentence in text:
    token_lists = lemmatizer_tagger(sentence)
    original.append(token_lists[0])
    lemmas.append(token_lists[1])
    tags.append(token_lists[2])
  return original, lemmas, tags

def coreference(text):
  '''
  Takes in text (str) and returns coreferenced text (str)
  '''
  nlp = spacy.load('en_core_web_sm')  # load the model
  neuralcoref.add_to_pipe(nlp)
  doc = nlp(text)  # get the spaCy Doc (composed of Tokens)
  return doc._.coref_clusters

def pipeline(file_path):
  '''
  Defines the total preprocessing + algorithm in a single pipeline. (Wrapper Function)

  Args:
    file_path (str): Path of the file (doc or txt or similar) containing text

  Output:
    Triples needed for knowledge graph
  '''
  # Read the text from the file
  with open(file_path, 'r', encoding='utf-8') as file:
      text = file.read()
  text = text.replace("Fig.", "Fig")
  preprocessed = preprocess_text(text)
  # coref_text = coreference(preprocessed)
  coref_text = preprocessed
  semantic_tagged = semantic_tag_text(coref_text)
  original, lemmas, tags = lem_tag_text(semantic_tagged)
  return original, lemmas, tags

In [ ]:
# Example usage:
nltk.download('wordnet')
input_file_path = "/content/drive/MyDrive/Attempt_1.txt"
print(pipeline(input_file_path))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


NameError: ignored

### **EXPERIMENTS**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a1= lesk(word_tokenize('This is the solution to the given question'),'solution')
print(a1,a1.definition())
a2 = lesk(word_tokenize('A solvent and solute together make up a solution'),'solution')
print(a2,a2.definition())

In [ ]:
import nltk
from nltk.corpus import wordnet

def lesk_algorithm(target_word, context):
    # Tokenize the context
    context_tokens = nltk.word_tokenize(context.lower())

    # Get WordNet synsets for the target word
    target_synsets = wordnet.synsets(target_word)

    best_sense = None
    max_overlap = 0

    for synset in target_synsets:
        # Get the definition of the synset
        synset_definition = synset.definition()
        # Tokenize the definition
        definition_tokens = nltk.word_tokenize(synset_definition.lower())

        # Compute the overlap between context and definition
        overlap = len(set(context_tokens) & set(definition_tokens))

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset

    return best_sense

# Example usage
target_word = "bank"  # The word for which you want to disambiguate the sense
context = "He deposited money in the bank."  # The context sentence

best_sense = lesk_algorithm(target_word, context)
print("Disambiguated sense:", best_sense.definition())


In [ ]:
import nltk
from nltk.corpus import wordnet

def get_word_definitions(word):
    synsets = wordnet.synsets(word)
    definitions = []

    for synset in synsets:
        definition = synset.definition()
        definitions.append(definition)

    return definitions

# Example usage
word = "table"
definitions = get_word_definitions(word)
for i, definition in enumerate(definitions, start=1):
    print(f"Definition {i}: {definition}")


In [ ]:
for token_lists in lemmatizer_tagger(sentence = "the bats saw the cats with best stripes hanging upside down by their feet"):
  print(token_lists)

['the', 'bats', 'saw', 'the', 'cats', 'with', 'best', 'stripes', 'hanging', 'upside', 'down', 'by', 'their', 'feet']
['the', 'bat', 'see', 'the', 'cat', 'with', 'good', 'stripe', 'hang', 'upside', 'down', 'by', 'their', 'foot']
['DT', 'NNS', 'VVD', 'DT', 'NNS', 'IN', 'JJS', 'NNS', 'VVG', 'RB', 'RB', 'IN', 'PP$', 'NNS']


In [ ]:
import spacy
import neuralcoref

nlp = spacy.load('en_core_web_sm')  # load the model
neuralcoref.add_to_pipe(nlp)

text = "Joseph Robinette Biden Jr. is an American politician who is the 46th and\
current president of the United States. A member of the Democratic Party, \
he served as the 47th vice president from 2009 to 2017 under Barack Obama and\
represented Delaware in the United States Senate from 1973 to 2009."

doc = nlp(text)  # get the spaCy Doc (composed of Tokens)

print(doc._.coref_clusters)  # You can see cluster of similar mentions

ModuleNotFoundError: ignored

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def CHUNKPHRASES(document):
    # Initialize the NLTK tokenizer and part-of-speech tagger
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    pos_tagger = nltk.pos_tag

    # Define chunking patterns
    grammar = r"""
        ENTITY: {<NN.*>+}             # Chunk sequences of nouns
                {<VB.*>+}             # Chunk sequences of verbs
                {<IN><DT>?<NN.*>+}    # Chunk prepositional phrases
                {<DT>?<JJ>*<NN.*>+}   # Chunk adjectival phrases
                {<VB.*><RP>}          # Chunk verb + particle
                {<VB.*><IN>}          # Chunk verb + adpositions
                {<IN><VB.*>}          # Chunk adpositions + verb
                {<RP><VB.*>}          # Chunk particle + verb
                {<VB.*><VB.*>}        # Chunk verb + verb
    """
    chunk_parser = nltk.RegexpParser(grammar)

    # Process each sentence in the document
    for sentence in document:
        words = tokenizer.tokenize(sentence)
        tagged_words = pos_tagger(words)
        chunks = chunk_parser.parse(tagged_words)

        # Extract phrases tagged as ENTITY
        for subtree in chunks.subtrees(filter=lambda t: t.label() == 'ENTITY'):
            phrases = [word for word, tag in subtree.leaves()]
            entity_phrase = ' '.join(phrases)
            print("ENTITY:", entity_phrase)

    return document

# Example usage
document = [
    "The black cat sat on the roof.",
    "She picked up the red apple.",
    "He ran quickly to the store.",
]

CHUNKPHRASES(document)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


ENTITY: cat
ENTITY: sat
ENTITY: roof
ENTITY: picked
ENTITY: apple
ENTITY: ran
ENTITY: store


['The black cat sat on the roof.',
 'She picked up the red apple.',
 'He ran quickly to the store.']